# Create a Slice with P4 tofino switch running a simple P4 program

In this notebook, we will create a network slice using a P4 Tofino switch and run a simple P4 program. We will connect two nodes through the P4 switch on the same network, enabling them to ping each other.

## Prerequisites
1. Access to a P4 Tofino switch.

2. Two nodes to connect through the switch.

3. Basic understanding of P4 programming.

4. Tofino Software Development Environment (SDE) installed.

In [ ]:
from IPython.display import Image

# Display the image
Image(filename='figs/Simple_P4_Lab.drawio.png')

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following script sets up two nodes, each with Shared NICs connected to two ports on a P4 tofino switch.

NIC component model options include:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports)
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)

In [ ]:
slice_name = 'P4-Lab-Slice'
p4_column_name = "p4-switch_available"

site1 = "LBNL"
site2 = "LBNL"
site3 = "RENC"

node1_name = 'Node1'
node2_name = 'Node2'
p4_name = 'P4'
network1_name = 'Network1'
network2_name = 'Network2'

print(f"Sites: {site1}, {site2}")
print(f"P4 Site: {site3}")

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Create Network
Network1 = slice.add_l2network(name=network1_name, subnet=IPv4Network("192.168.0.0/24"))
Network2 = slice.add_l2network(name=network2_name, subnet=IPv4Network("192.168.0.0/24"))

# Create Node 1 and its links
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]
iface1.set_mode('config')
Network1.add_interface(iface1)
iface1.set_ip_addr(IPv4Address("192.168.0.1"))

# Create P4 switch and its links 
p4 = slice.add_switch(name=p4_name, site=site3)
iface2 = p4.get_interfaces()[0]
Network1.add_interface(iface2)

iface3 = p4.get_interfaces()[1]
Network2.add_interface(iface3)

# Create Node 2 and its links
node2 = slice.add_node(name=node2_name, site=site2)
iface4 = node2.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]
iface4.set_mode('config')
Network2.add_interface(iface4)
iface4.set_ip_addr(IPv4Address("192.168.0.2"))

# Submit Slice Request
slice.submit()

## Run the Experiment

### Configure the P4 Switch. Login locally on P4 switch and perform below commands.

- Go to Lab1 and source config environment
```
cd ~/P4_labs/lab1/
source config_env.sh
```
- Compile the code
```
~/tools/p4_build.sh --with-p4c=bf-p4c p4src/basic.p4
```
- Run the code
```
cd ~/bf-sde-9.13.3
./run_switchd.sh -p basic
```
- Enable Ports on bfshell
```
bf-shell> ucli
bf-sde> pm port-add 1/- 100G NONE
bf-sde> pm port-add 2/- 100G NONE
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|128|2/ 0|100G   | RS |Au|Au|YES|DIS|DWN|  NONE  |               0|               0|
2/0  |22/0|136|2/ 8|100G   | RS |Au|Au|YES|DIS|DWN|  NONE  |               0|               0|
bf-sde> pm port-enb 1/-
bf-sde> pm port-enb 2/-
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|128|2/ 0|100G   | RS |Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
2/0  |22/0|136|2/ 8|100G   | RS |Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
bf-sde>
```

- Set up the forwarding rules on another window:
  - Update the port numbers used in `~/P4_labs/lab1/bfrt_python/setup.py` are same as indicated by D_P column above 
  - Run the commands below
```
cd ~/bf-sde-9.13.3; . ~/tools/set_sde.bash
$SDE/./run_bfshell.sh --no-status-srv -b ~/P4_labs/lab1/bfrt_python/setup.py
```

In [ ]:
from IPython.display import Image

# Display the image
Image(filename='figs/P4-Lab-Slice.png')

### Verifying Reachability through Ping Tests

In [ ]:
slice=fablib.get_slice(slice_name)
node1=slice.get_node(node1_name)
node2=slice.get_node(node2_name)

node1_addr = node1.get_interface(network_name=network1_name).get_ip_addr()
node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
stdout, stderr = node2.execute(f'ping -c 5 {node1_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice=fablib.get_slice(slice_name)
slice.delete()